### Test Connection 

In [71]:
from openai import OpenAI
from nemo_curator import OpenAIClient
from nemo_curator.synthetic import NemotronGenerator

openai_client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key="NVIDIA_API_KEY"
)
client = OpenAIClient(openai_client)
generator = NemotronGenerator(client)

n_macro_topics = 20
model = "nvdev/nvidia/llama-3.1-nemotron-70b-instruct"
model_kwargs = {
    "temperature": 0.2,
    "top_p": 0.7,
    "max_tokens": 1024,
}

responses = generator.generate_macro_topics(
    n_macro_topics=n_macro_topics, model=model, model_kwargs=model_kwargs
)

print(responses[0])
# Output:
# 1. Climate Change and Sustainable Living
# 2. Space Exploration and the Universe
# ...

Below is a list of 20 comprehensive topics that cover a wide range of aspects of our daily life, the world, and science, ensuring diversity across different domains:

1. **Sustainable Living Practices** - Exploring eco-friendly habits for a greener future.
2. **Global Economic Trends** - Analyzing shifts in the world economy and their impacts.
3. **Mental Health in the Digital Age** - Understanding the intersection of technology and mental wellness.
4. **Renewable Energy Innovations** - Delving into the latest in solar, wind, and other renewable energy technologies.
5. **Cultural Heritage Preservation** - Efforts to protect historical sites and traditions worldwide.
6. **The Future of Space Exploration** - Missions, discoveries, and the quest for life beyond Earth.
7. **Nutrition and Dietetics for Wellbeing** - Science-backed dietary advice for optimal health.
8. **Cybersecurity Threats and Solutions** - Protecting personal and corporate data in the digital era.
9. **Biodiversity and E

### Test Question Generation

In [ ]:
from openai import OpenAI
from nemo_curator import OpenAIClient

openai_client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key="NVIDIA_API_KEY",
)
client = OpenAIClient(openai_client)
responses = client.query_model(
    model="nvdev/nvidia/llama-3.1-nemotron-70b-instruct",
    messages=[
        {
            "role": "user",
            "content": "Write a limerick about the wonders of GPU computing.",
        }
    ],
    temperature=0.2,
    top_p=0.7,
    max_tokens=1024,
)
print(responses[0])
# Output:
# A GPU with numbers in flight, Brings joy to programmers late at night.
# With parallel delight, Solving problems, so bright,
# In the realm of computing, it's quite a sight!

Here is a limerick about the wonders of GPU computing:

There once was a GPU so fine,
Whose parallel processing did shine.
It crunched with great zest,
Through complex tests,
And made computations divine.


### Test Reward Score Generation

In [1]:
from openai import OpenAI
from nemo_curator import OpenAIClient

openai_client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key="NVIDIA_API_KEY",
)
client = OpenAIClient(openai_client)

model = "nvdev/nvidia/llama-3.1-nemotron-70b-reward"

messages = [
    {"role": "user", "content": "I am going to Paris, what should I see?"},
    {
        "role": "assistant",
        "content": "Ah, Paris, the City of Light! There are so many amazing things to see and do in this beautiful city ...",
    },
]

rewards = client.query_reward_model(messages=messages, model=model)
print(rewards)
# {
# "helpfulness": 1.6171875
# "correctness": 1.6484375
# "coherence": 3.3125
# "complexity": 0.546875
# "verbosity": 0.515625
# }

-19.875


### Import ragas_dataset

In [3]:
import pandas as pd

ragas = pd.read_csv('ragas_dataset.csv')
ragas.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the importance of including a module d...,[' appropriate best practice file naming conve...,NaN,simple,[{'source': 'https://lilianweng.github.io/post...,True
1,How does the response generation stage in Hugg...,['gingFace platform according to the model des...,NaN,simple,[{'source': 'https://lilianweng.github.io/post...,True
2,What are some common limitations faced when de...,[' code:\\n\\nFILENAME\\n```LANG\\nCODE\\n```\...,Some common limitations faced when developing ...,simple,[{'source': 'https://lilianweng.github.io/post...,True
3,How does Algorithm Distillation (AD) improve t...,[' encapsulated in a long history-conditioned ...,Algorithm Distillation (AD) improves the learn...,simple,[{'source': 'https://lilianweng.github.io/post...,True
4,What are the key components and functionalitie...,['\n\n\n\n\n\nLLM Powered Autonomous Agents | ...,The key components and functionalities of LLM-...,simple,[{'source': 'https://lilianweng.github.io/post...,True


### Generate NeMo Questions using Ragas Contexts  

In [130]:
ragas.columns

Index(['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata',
       'episode_done', 'sdg_nemotron_q', 'sdg_nemotron_a',
       'sdg_nemotron_reward', 'sdg_nemotron_ragas_a'],
      dtype='object')

In [146]:
model = "nvdev/nvidia/llama-3.1-nemotron-70b-instruct"

question_lst = []

for i in range(len(ragas)):
    closed_qa_responses = client.query_model(
        model=model,
        messages = [
            {
                "role": "user",
                "content": f"Generate a single, concise question similar to '{ragas.question[i]}' based on the provided context. Do NOT include any explanations, rationale, or additional text in your response—just the question itself: {ragas.contexts[i]}"
            }
        ],
        temperature = 0.2,
        top_p = 0.7,
        max_tokens = 1024,
    )
    
    question = closed_qa_responses[0]
    question_lst.append(question)
    
ragas['sdg_nemotron_q'] = question_lst

What is the significance of adhering to a language-specific best practice for file naming conventions when organizing multiple classes and functions across different files in a project?
How does the task execution stage in HuggingGPT address challenges related to model inference latency, expert model selection accuracy, and result interpretation complexity?
What are the primary architectural trade-offs when designing LLM-powered autonomous agent systems to mitigate limitations in context length, long-term planning, and natural language interface reliability?
How does Algorithm Distillation (AD) leverage multi-episode learning histories to enhance in-context reinforcement learning, compared to baselines like Expert Distillation (ED) and RL^2?
What are the core architectural elements and operational workflows of LLM-powered autonomous agents that enable effective planning, memory utilization, and tool integration?
How do consistent naming conventions, modular imports, and dependency defi

### Generate NeMo Answers

In [160]:
model = "nvdev/nvidia/llama-3.1-nemotron-70b-instruct"

answer_lst = []

for i in range(len(ragas)):
    responses = client.query_model(
        model=model,
        messages=[
            {
                "role": "user",
                "content": f"Generate a single, concise answer to '{ragas.sdg_nemotron_q[i]}' based on the provided context. Do NOT include any explanations, rationale, or additional text or special character patterns in your response—just the answer itself: {ragas.contexts[i]}"
             }
        ],
        temperature=0.2,
        top_p=0.7,
        max_tokens=1024,
    )
    
    answer = responses[0]
    answer_lst.append(answer)

ragas['sdg_nemotron_a'] = answer_lst 

In [161]:
for i in range(10):
    print(i, '.')
    print(question_lst[i])
    print(answer_lst[i])
    print('-----------------------------------------------------------------------------------------------------------------------------------------------------------')

0 .
What is the significance of adhering to a language-specific best practice for file naming conventions when organizing multiple classes and functions across different files in a project?
Ensures maintainability, compatibility, readability, and scalability by providing a consistent structure, facilitating collaboration, and enabling efficient dependency management across the project.
-----------------------------------------------------------------------------------------------------------------------------------------------------------
1 .
How does the task execution stage in HuggingGPT address challenges related to model inference latency, expert model selection accuracy, and result interpretation complexity?
**Task Execution Stage in HuggingGPT Addresses Challenges as Follows:**

* **Model Inference Latency:** Expert models execute specific tasks in parallel, reducing overall latency.
* **Expert Model Selection Accuracy:** LLM frames task as a multiple-choice question, ensuring ac

### Generate Reward Score for NeMo Questions and Answers

In [167]:
reward_lst = []

model = "nvdev/nvidia/llama-3.1-nemotron-70b-reward"

for i in range(len(ragas)):
    messages = [
        {
            "role": "user",
            "content": f"""
                            Provide a concise and well-reasoned answer to the following question:
                            "{ragas.sdg_nemotron_q[i]}"
                            Use the context below to ensure accuracy and clarity:
                            {ragas.contexts[i]}
                        """
        },
        {
            "role": "assistant",
            "content": ragas.sdg_nemotron_a[i],
        },
    ]

    
    rewards = client.query_reward_model(messages=messages, model=model)
    rewards = float(rewards)
    
    reward_lst.append(rewards)
    
ragas['sdg_nemotron_reward'] = reward_lst 

In [170]:
print(ragas.ground_truth[i])
import numpy as np

Context limits and task challenges affect LLM agent reliability by restricting the inclusion of historical information and detailed instructions due to finite context length. This limitation impacts long-term planning and task decomposition, making it difficult for LLMs to adjust plans when faced with unexpected errors. Additionally, the reliance on natural language interfaces can lead to formatting errors and unreliable outputs, further affecting the reliability of LLM agents.


### Generate NeMo Answers for Ragas Questions

In [171]:
model = "nvdev/nvidia/llama-3.1-nemotron-70b-instruct"

nemo_answer_lst = []

for i in range(len(ragas)):
    responses = client.query_model(
        model=model,
        messages=[
            {
                "role": "user",
                "content": f"Generate a single, concise answer to '{ragas.question[i]}' based on the provided context. Do NOT include any explanations, rationale, or additional text or special character patterns in your response—just the answer itself: {ragas.contexts[i]}"
             }
        ],
        temperature=0.2,
        top_p=0.7,
        max_tokens=1024,
    )

    response = responses[0]
    nemo_answer_lst.append(response)

ragas['sdg_nemotron_ragas_a'] = nemo_answer_lst 

In [179]:
print(ragas[ragas.ground_truth != '']['ground_truth'][4])
print('----------------------------------')
print(ragas[ragas.ground_truth != '']['sdg_nemotron_ragas_a'][4])

The key components and functionalities of LLM-powered autonomous agents include: 1) Planning: This involves task decomposition, where large tasks are broken down into smaller, manageable subgoals, and self-reflection, where the agent learns from past actions to improve future results. 2) Memory: This includes short-term memory for in-context learning and long-term memory for retaining and recalling information over extended periods. 3) Tool Use: The agent learns to call external APIs for additional information, code execution, and access to proprietary information sources.
----------------------------------
**Key Components and Functionalities of LLM-Powered Autonomous Agents:**

1. **Planning**
	* Task Decomposition (subgoal identification, step-by-step breakdown)
	* Self-Reflection (self-criticism, refinement of past actions for improvement)
2. **Memory**
	* Short-term Memory (in-context learning, prompt engineering)
	* Long-term Memory (external vector store for retaining and recall

### Export the reuslts to .csv file

In [180]:
ragas.to_csv('ragas_nemotron_comparison.csv')